In [10]:
import scipy.sparse as sp
import numpy as np
import json
from pathlib import Path
import networkx as nx
path = Path('data_2024')
adj = sp.load_npz(path/'adj.npz')
feat  = np.load(path/'features.npy')
labels = np.load(path/'labels.npy')
splits = json.load(open(path/'splits.json'))
idx_train, idx_test = splits['idx_train'], splits['idx_test']

In [11]:
torch.cuda.is_available()
torch.cuda.device_count()

4

In [12]:
idx_test

[2119,
 2206,
 2165,
 1994,
 520,
 521,
 2103,
 1838,
 1840,
 116,
 134,
 2477,
 1522,
 821,
 1393,
 1983,
 2136,
 2478,
 1182,
 1617,
 899,
 1898,
 1592,
 903,
 815,
 1395,
 871,
 1596,
 1188,
 513,
 791,
 1296,
 1608,
 1149,
 1568,
 1625,
 1101,
 829,
 1595,
 1912,
 1856,
 308,
 603,
 1122,
 566,
 2175,
 2066,
 1556,
 339,
 1264,
 531,
 2266,
 3,
 191,
 423,
 2054,
 1098,
 338,
 2215,
 2184,
 302,
 982,
 725,
 362,
 1931,
 2178,
 745,
 1248,
 376,
 171,
 646,
 1569,
 1778,
 1974,
 321,
 156,
 2265,
 1814,
 1050,
 868,
 846,
 1969,
 2382,
 1508,
 636,
 428,
 2164,
 1044,
 1378,
 2259,
 1265,
 809,
 2387,
 1056,
 268,
 1459,
 1509,
 1158,
 1107,
 1024,
 755,
 293,
 2218,
 194,
 770,
 661,
 757,
 1450,
 1328,
 1151,
 986,
 1256,
 325,
 827,
 1300,
 1890,
 607,
 76,
 292,
 2237,
 236,
 1474,
 203,
 1065,
 2317,
 724,
 2308,
 2236,
 100,
 1423,
 1824,
 140,
 1641,
 148,
 1672,
 1540,
 240,
 2145,
 364,
 1291,
 921,
 637,
 1085,
 1484,
 139,
 1587,
 758,
 974,
 1257,
 1476,
 43,
 878,
 226

In [13]:
from torch_geometric.utils import from_scipy_sparse_matrix
edge_index = from_scipy_sparse_matrix(adj)
edge_index

(tensor([[   0,    0,    0,  ..., 2478, 2478, 2479],
         [1084, 1104, 1288,  ...,  931,  933,  999]]),
 tensor([1., 1., 1.,  ..., 1., 1., 1.]))

In [14]:
labels.shape

(496,)

In [15]:
feat.shape

(2480, 1390)

In [16]:
len(splits['idx_train']), len(splits['idx_test'])

(496, 1984)

In [17]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

data = Data(x=torch.tensor(feat, dtype=torch.float),
            edge_index=edge_index[0],
            y=torch.tensor(labels, dtype=torch.long))

In [18]:
data

Data(x=[2480, 1390], edge_index=[2, 10100], y=[496])

In [19]:
data.num_nodes

2480

In [23]:
data.train_mask.shape

torch.Size([2480])

In [32]:
data.train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.train_mask[idx_train] = 1
data.test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.test_mask[idx_test] = 1

In [35]:
data.y[data.train_mask == True]

IndexError: The shape of the mask [2480] at index 0 does not match the shape of the indexed tensor [496] at index 0

In [36]:
data.train_mask

tensor([ True, False,  True,  ..., False, False, False])

In [12]:
data

Data(x=[2480, 1390], edge_index=[2, 10100], y=[496], train_mask=[2480], test_mask=[2480])

In [37]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from torch_geometric.nn import APPNP

class GraphSage(torch.nn.Module):
    def __init__(self, num_node_features, num_hidden, num_classes):
        super().__init__()
        self.appnp = APPNP(K=100, alpha=0.1, dropout=0, cached=True, add_self_loops=True, normalize=False)
        self.conv1 = SAGEConv(num_node_features, num_hidden)
       # self.conv2 = SAGEConv(num_hidden, num_hidden)
       # self.conv3 = SAGEConv(num_hidden, num_hidden)
       # self.conv4 = SAGEConv(num_hidden, num_hidden)
       # self.conv5 = SAGEConv(num_hidden, num_hidden)
       # self.conv6 = SAGEConv(num_hidden, num_hidden)
       # self.conv7 = SAGEConv(num_hidden, num_hidden)
        self.conv2 = SAGEConv(num_hidden, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.message_passing(x, edge_index, self.conv1)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.message_passing(x, edge_index, self.conv2)
      #  x = F.relu(x)
        # x = F.dropout(x, training=self.training)
        # x = self.message_passing(x, edge_index, self.conv3)
        # x = F.relu(x)
        # x = F.dropout(x, training=self.training)
        # x = self.message_passing(x, edge_index, self.conv4)
        # x = F.relu(x)
        # x = F.dropout(x, training=self.training)
        # x = self.message_passing(x, edge_index, self.conv5)
        # x = F.relu(x)
        # x = F.dropout(x, training=self.training)
        # x = self.message_passing(x, edge_index, self.conv6)
        # x = F.relu(x)
        # x = F.dropout(x, training=self.training)
        # x = self.message_passing(x, edge_index, self.conv7)
        # x = F.relu(x)
        # x = F.dropout(x, training=self.training)
        # x = self.message_passing(x, edge_index, self.conv8)
        
        return F.log_softmax(x, dim=1)
    
    def message_passing(self, x, edge_index, conv):
        # Perform message passing with the given convolutional layer
        return conv(x, edge_index)



In [27]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from torch_geometric.nn import APPNP

class GraphSage2(torch.nn.Module):
    def __init__(self, num_node_features, num_hidden, num_classes):
        super().__init__()
        self.appnp = APPNP(K=100, alpha=0.1, dropout=0, cached=True, add_self_loops=True, normalize=False)
        self.conv1 = SAGEConv(num_node_features, num_hidden)
        self.conv2 = SAGEConv(num_hidden, num_hidden)
        self.conv3 = SAGEConv(num_hidden, num_hidden)
        self.conv4 = SAGEConv(num_hidden, num_hidden)
        self.conv5 = SAGEConv(num_hidden, num_hidden)
        self.conv6 = SAGEConv(num_hidden, num_hidden)
        self.conv7 = SAGEConv(num_hidden, num_hidden)
        self.conv8 = SAGEConv(num_hidden, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.message_passing(x, edge_index, self.conv1)
        x = F.tanh(x)
        x = F.dropout(x, training=self.training)
        x = self.message_passing(x, edge_index, self.conv2)
        x = F.tanh(x)
        x = F.dropout(x, training=self.training)
        x = self.message_passing(x, edge_index, self.conv3)
        x = F.tanh(x)
        x = F.dropout(x, training=self.training)
        x = self.message_passing(x, edge_index, self.conv4)
        x = F.tanh(x)
        x = F.dropout(x, training=self.training)
        x = self.message_passing(x, edge_index, self.conv5)
        x = F.tanh(x)
        x = F.dropout(x, training=self.training)
        x = self.message_passing(x, edge_index, self.conv6)
        x = F.tanh(x)
        x = F.dropout(x, training=self.training)
        x = self.message_passing(x, edge_index, self.conv7)
        x = F.tanh(x)
        x = F.dropout(x, training=self.training)
        x = self.message_passing(x, edge_index, self.conv8)
        
        return F.log_softmax(x, dim=1)
    
    def message_passing(self, x, edge_index, conv):
        # Perform message passing with the given convolutional layer
        return conv(x, edge_index)


In [28]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
r

0

In [31]:
train_mask

NameError: name 'train_mask' is not defined

In [29]:
train_idx, test_idx = train_test_split(np.arange(len(data.y)), test_size=0.4, random_state=9)

NameError: name 'train_test_split' is not defined

In [24]:
train_idx

NameError: name 'train_idx' is not defined

In [41]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from sklearn.model_selection import train_test_split, KFold
import numpy as np

best_val_acc = 0.0
best_model = None
best_test_acc = 0.0
best_test_predictions = None

train_idx, test_idx = train_test_split(np.arange(len(data.y)), test_size=0.4, random_state=9)

k_folds = 10
kf = KFold(n_splits=k_folds, shuffle=False)

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')


accuracies = []
test_accuracies = []

for fold, (train_fold_idx, val_idx) in enumerate(kf.split(train_idx)):
    print(f"Fold {fold + 1}/{k_folds}")

    data = data.to(device)

    model = GraphSage(num_node_features=data.x.shape[1],
                      num_hidden=128,
                      num_classes=(data.y.max() + 1).item()
                      ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=5e-3)

    best_epoch = 0
    for epoch in range(200):
        model.train()
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[train_idx[train_fold_idx]], data.y[train_idx[train_fold_idx]])
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            pred = model(data).argmax(dim=1)
            val_acc = pred[train_idx[val_idx]].eq(data.y[train_idx[val_idx]]).sum().item() / len(val_idx)

            if val_acc > best_val_acc:
                best_val_acc = val_acc
                best_epoch = epoch
                best_model = model.state_dict().copy()

    print(f"Best Validation Accuracy: {best_val_acc} at Epoch {best_epoch}")

    # Load the best model for inference
    model.load_state_dict(best_model)
    model.eval()
    with torch.no_grad():
        pred = model(data).argmax(dim=1)
        acc = pred[test_idx].eq(data.y[test_idx]).sum().item() / len(test_idx)
        print(f"Test Accuracy: {acc}")
        if acc > best_test_acc:
            best_test_acc = acc
            best_test_predictions = pred[idx_test]
           # np.savetxt('submissiontest.txt', best_test_predictions, fmt='%d')
print(f"Best Validation Accuracy: {best_val_acc} at Epoch {best_epoch}")
print(f"Best Test Accuracy: {acc}")

Fold 1/10
Best Validation Accuracy: 0.3333333333333333 at Epoch 157
Test Accuracy: 0.2814070351758794
Fold 2/10
Best Validation Accuracy: 0.3333333333333333 at Epoch 0
Test Accuracy: 0.2814070351758794
Fold 3/10
Best Validation Accuracy: 0.4 at Epoch 164
Test Accuracy: 0.2964824120603015
Fold 4/10
Best Validation Accuracy: 0.4 at Epoch 0
Test Accuracy: 0.2964824120603015
Fold 5/10
Best Validation Accuracy: 0.4 at Epoch 0
Test Accuracy: 0.2964824120603015
Fold 6/10
Best Validation Accuracy: 0.4 at Epoch 0
Test Accuracy: 0.2964824120603015
Fold 7/10
Best Validation Accuracy: 0.4 at Epoch 0
Test Accuracy: 0.2964824120603015
Fold 8/10
Best Validation Accuracy: 0.4 at Epoch 0
Test Accuracy: 0.2964824120603015
Fold 9/10
Best Validation Accuracy: 0.4 at Epoch 0
Test Accuracy: 0.2964824120603015
Fold 10/10
Best Validation Accuracy: 0.4 at Epoch 0
Test Accuracy: 0.2964824120603015


In [42]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from sklearn.model_selection import train_test_split, KFold
import numpy as np

best_val_acc = 0.0
best_model = None
best_test_acc = 0.0
best_test_predictions = None

train_idx, test_idx = train_test_split(np.arange(len(data.y)), test_size=0.4, random_state=9)

k_folds = 20
kf = KFold(n_splits=k_folds, shuffle=False)

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')


accuracies = []
test_accuracies = []

for fold, (train_fold_idx, val_idx) in enumerate(kf.split(train_idx)):
    print(f"Fold {fold + 1}/{k_folds}")

    data = data.to(device)

    model = GraphSage2(num_node_features=data.x.shape[1],
                      num_hidden=64,
                      num_classes=(data.y.max() + 1).item()
                      ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=5e-3)

    best_epoch = 0
    for epoch in range(200):
        model.train()
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[train_idx[train_fold_idx]], data.y[train_idx[train_fold_idx]])
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            pred = model(data).argmax(dim=1)
            val_acc = pred[train_idx[val_idx]].eq(data.y[train_idx[val_idx]]).sum().item() / len(val_idx)

            if val_acc > best_val_acc:
                best_val_acc = val_acc
                best_epoch = epoch
                best_model = model.state_dict().copy()

    print(f"Best Validation Accuracy: {best_val_acc} at Epoch {best_epoch}")

    # Load the best model for inference
    model.load_state_dict(best_model)
    model.eval()
    with torch.no_grad():
        pred = model(data).argmax(dim=1)
        acc = pred[test_idx].eq(data.y[test_idx]).sum().item() / len(test_idx)
        print(f"Test Accuracy: {acc}")
        if acc > best_test_acc:
            best_test_acc = acc
            best_test_predictions = pred[idx_test]
           # np.savetxt('submissiontest.txt', best_test_predictions, fmt='%d')


Fold 1/20
Best Validation Accuracy: 0.26666666666666666 at Epoch 181
Test Accuracy: 0.24120603015075376
Fold 2/20
Best Validation Accuracy: 0.4666666666666667 at Epoch 16
Test Accuracy: 0.271356783919598
Fold 3/20
Best Validation Accuracy: 0.4666666666666667 at Epoch 0
Test Accuracy: 0.271356783919598
Fold 4/20
Best Validation Accuracy: 0.4666666666666667 at Epoch 0
Test Accuracy: 0.271356783919598
Fold 5/20
Best Validation Accuracy: 0.4666666666666667 at Epoch 0
Test Accuracy: 0.271356783919598
Fold 6/20
Best Validation Accuracy: 0.4666666666666667 at Epoch 0
Test Accuracy: 0.271356783919598
Fold 7/20
Best Validation Accuracy: 0.4666666666666667 at Epoch 0
Test Accuracy: 0.271356783919598
Fold 8/20
Best Validation Accuracy: 0.4666666666666667 at Epoch 0
Test Accuracy: 0.271356783919598
Fold 9/20
Best Validation Accuracy: 0.4666666666666667 at Epoch 0
Test Accuracy: 0.271356783919598
Fold 10/20
Best Validation Accuracy: 0.4666666666666667 at Epoch 0
Test Accuracy: 0.271356783919598
Fol

In [14]:
from torch_geometric.nn import GATConv
from torch_geometric.nn import APPNP

class GAT(torch.nn.Module):
    def __init__(self, num_node_features, num_hidden, num_classes):
        super().__init__()
        self.conv1 = GATConv(num_node_features, num_hidden, heads=64, dropout=0.4)
        self.conv2 = GATConv(num_hidden * 64, num_classes, heads=1, concat=True, dropout=0.4)
        self.appnp = APPNP(K=100, alpha=0.1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.appnp(x, edge_index)
        return F.log_softmax(x, dim=1)
    
    def message_passing(self, x, edge_index, conv):
        # Perform message passing with the given convolutional layer
        return conv(x, edge_index)


In [19]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from sklearn.model_selection import train_test_split, KFold
import numpy as np

best_val_acc = 0.0
best_model = None
best_test_acc = 0.0
best_test_predictions = None

train_idx, test_idx = train_test_split(np.arange(len(data.y)), test_size=0.3, random_state=123)

k_folds = 10
kf = KFold(n_splits=k_folds, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

accuracies = []
test_accuracies = []

for fold, (train_fold_idx, val_idx) in enumerate(kf.split(train_idx)):
    print(f"Fold {fold + 1}/{k_folds}")

    data = data.to(device)

    model = GAT(num_node_features=data.x.shape[1],
                      num_hidden=150,
                      num_classes=(data.y.max() + 1).item()
                      ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, weight_decay=5e-4)

    best_epoch = 0
    for epoch in range(100):
        model.train()
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[train_idx[train_fold_idx]], data.y[train_idx[train_fold_idx]])
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            pred = model(data).argmax(dim=1)
            val_acc = pred[train_idx[val_idx]].eq(data.y[train_idx[val_idx]]).sum().item() / len(val_idx)

            if val_acc > best_val_acc:
                best_val_acc = val_acc
                best_epoch = epoch
                best_model = model.state_dict().copy()

    print(f"Best Validation Accuracy: {best_val_acc} at Epoch {best_epoch}")

    # Load the best model for inference
    model.load_state_dict(best_model)
    model.eval()
    with torch.no_grad():
        pred = model(data).argmax(dim=1)
        acc = pred[test_idx].eq(data.y[test_idx]).sum().item() / len(test_idx)
        print(f"Test Accuracy: {acc}")
        if acc > best_test_acc:
            best_test_acc = acc
            best_test_predictions = pred[idx_test]
          #  np.savetxt('submissiontest.txt', best_test_predictions, fmt='%d')


Fold 1/5
Best Validation Accuracy: 0.3142857142857143 at Epoch 26
Test Accuracy: 0.2953020134228188
Fold 2/5
Best Validation Accuracy: 0.34285714285714286 at Epoch 93
Test Accuracy: 0.2953020134228188
Fold 3/5
Best Validation Accuracy: 0.34285714285714286 at Epoch 0
Test Accuracy: 0.2953020134228188
Fold 4/5
Best Validation Accuracy: 0.34285714285714286 at Epoch 0
Test Accuracy: 0.2953020134228188
Fold 5/5
Best Validation Accuracy: 0.34285714285714286 at Epoch 0
Test Accuracy: 0.2953020134228188


In [43]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from torch_geometric.nn import APPNP
from sklearn.model_selection import train_test_split, KFold
import numpy as np
import itertools
from tqdm import tqdm

class GraphSage(torch.nn.Module):
    def __init__(self, num_node_features, num_hidden, num_classes):
        super().__init__()
        self.appnp = APPNP(K=100, alpha=0.1, dropout=0, cached=True, add_self_loops=True, normalize=False)
        self.conv1 = SAGEConv(num_node_features, num_hidden)
        self.conv2 = SAGEConv(num_hidden, num_hidden)
        self.conv3 = SAGEConv(num_hidden, num_hidden)
        self.conv4 = SAGEConv(num_hidden, num_hidden)
        self.conv5 = SAGEConv(num_hidden, num_hidden)
        self.conv6 = SAGEConv(num_hidden, num_hidden)
        self.conv7 = SAGEConv(num_hidden, num_hidden)
        self.conv8 = SAGEConv(num_hidden, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.message_passing(x, edge_index, self.conv1)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.message_passing(x, edge_index, self.conv2)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.message_passing(x, edge_index, self.conv3)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.message_passing(x, edge_index, self.conv4)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.message_passing(x, edge_index, self.conv5)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.message_passing(x, edge_index, self.conv6)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.message_passing(x, edge_index, self.conv7)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.message_passing(x, edge_index, self.conv8)
        
        return F.log_softmax(x, dim=1)
    
    def message_passing(self, x, edge_index, conv):
        # Perform message passing with the given convolutional layer
        return conv(x, edge_index)


hyperparameters = {
    'num_hidden': [32, 64, 128],
    'lr': [0.0001, 0.001, 0.01],
    'weight_decay': [5e-4, 5e-3, 5e-2],
}

best_hyperparameters = None
best_val_acc = 0.0
best_val_acc = 0.0
best_model = None
best_test_acc = 0.0
best_test_predictions = None

total_iterations = len(hyperparameters['num_hidden']) * len(hyperparameters['lr']) * len(hyperparameters['weight_decay'])

pbar = tqdm(total=total_iterations, desc="Grid Search Progress")
for num_hidden, lr, weight_decay in itertools.product(*hyperparameters.values()):
    pbar.update(1)


  
    train_idx, test_idx = train_test_split(np.arange(len(data.y)), test_size=0.4, random_state=9)

    k_folds = 20
    kf = KFold(n_splits=k_folds, shuffle=False)

    device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')


    accuracies = []
    test_accuracies = []

    for fold, (train_fold_idx, val_idx) in enumerate(kf.split(train_idx)):
        print(f"Fold {fold + 1}/{k_folds}")

        data = data.to(device)

        model = GraphSage(num_node_features=data.x.shape[1],
                        num_hidden=num_hidden,
                        num_classes=(data.y.max() + 1).item()
                        ).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

        best_epoch = 0
        for epoch in range(200):
            model.train()
            optimizer.zero_grad()
            out = model(data)
            loss = F.nll_loss(out[train_idx[train_fold_idx]], data.y[train_idx[train_fold_idx]])
            loss.backward()
            optimizer.step()

            model.eval()
            with torch.no_grad():
                pred = model(data).argmax(dim=1)
                val_acc = pred[train_idx[val_idx]].eq(data.y[train_idx[val_idx]]).sum().item() / len(val_idx)

                if val_acc > best_val_acc:
                    best_val_acc = val_acc
                    best_epoch = epoch
                    best_hyperparameters = {'num_hidden': num_hidden, 'lr': lr, 'weight_decay': weight_decay}
                    best_model = model.state_dict().copy()
                    

        

        print(f"Best Validation Accuracy: {best_val_acc} at Epoch {best_epoch}")
        print("Best hyperparameters:", best_hyperparameters)

        # Load the best model for inference
        model.load_state_dict(best_model)
        model.eval()
        with torch.no_grad():
            pred = model(data).argmax(dim=1)
            acc = pred[test_idx].eq(data.y[test_idx]).sum().item() / len(test_idx)
            print(f"Test Accuracy: {acc}")
            if acc > best_test_acc:
                best_test_acc = acc
                best_test_predictions = pred[idx_test]
            # np.savetxt('submissiontest.txt', best_test_predictions, fmt='%d')



Grid Search Progress:   0%|          | 0/27 [00:00<?, ?it/s]

Fold 1/20
Best Validation Accuracy: 0.26666666666666666 at Epoch 128
Best hyperparameters: {'num_hidden': 32, 'lr': 0.0001, 'weight_decay': 0.0005}
Test Accuracy: 0.3015075376884422
Fold 2/20
Best Validation Accuracy: 0.3333333333333333 at Epoch 24
Best hyperparameters: {'num_hidden': 32, 'lr': 0.0001, 'weight_decay': 0.0005}
Test Accuracy: 0.3015075376884422
Fold 3/20
Best Validation Accuracy: 0.3333333333333333 at Epoch 0
Best hyperparameters: {'num_hidden': 32, 'lr': 0.0001, 'weight_decay': 0.0005}
Test Accuracy: 0.3015075376884422
Fold 4/20
Best Validation Accuracy: 0.4 at Epoch 107
Best hyperparameters: {'num_hidden': 32, 'lr': 0.0001, 'weight_decay': 0.0005}
Test Accuracy: 0.3015075376884422
Fold 5/20
Best Validation Accuracy: 0.4 at Epoch 0
Best hyperparameters: {'num_hidden': 32, 'lr': 0.0001, 'weight_decay': 0.0005}
Test Accuracy: 0.3015075376884422
Fold 6/20
Best Validation Accuracy: 0.4 at Epoch 0
Best hyperparameters: {'num_hidden': 32, 'lr': 0.0001, 'weight_decay': 0.0005}

Grid Search Progress:   7%|▋         | 2/27 [00:34<07:14, 17.37s/it]

Best Validation Accuracy: 0.5 at Epoch 100
Best hyperparameters: {'num_hidden': 32, 'lr': 0.0001, 'weight_decay': 0.0005}
Test Accuracy: 0.3015075376884422
Fold 1/20
Best Validation Accuracy: 0.5 at Epoch 0
Best hyperparameters: {'num_hidden': 32, 'lr': 0.0001, 'weight_decay': 0.0005}
Test Accuracy: 0.3015075376884422
Fold 2/20
Best Validation Accuracy: 0.5 at Epoch 0
Best hyperparameters: {'num_hidden': 32, 'lr': 0.0001, 'weight_decay': 0.0005}
Test Accuracy: 0.3015075376884422
Fold 3/20
Best Validation Accuracy: 0.5 at Epoch 0
Best hyperparameters: {'num_hidden': 32, 'lr': 0.0001, 'weight_decay': 0.0005}
Test Accuracy: 0.3015075376884422
Fold 4/20
Best Validation Accuracy: 0.5 at Epoch 0
Best hyperparameters: {'num_hidden': 32, 'lr': 0.0001, 'weight_decay': 0.0005}
Test Accuracy: 0.3015075376884422
Fold 5/20
Best Validation Accuracy: 0.5 at Epoch 0
Best hyperparameters: {'num_hidden': 32, 'lr': 0.0001, 'weight_decay': 0.0005}
Test Accuracy: 0.3015075376884422
Fold 6/20
Best Validatio

Grid Search Progress:  11%|█         | 3/27 [01:18<11:20, 28.35s/it]

...

Grid Search Progress:  33%|███▎      | 9/27 [05:48<13:00, 43.36s/it]

Best Validation Accuracy: 0.5333333333333333 at Epoch 0
Best hyperparameters: {'num_hidden': 32, 'lr': 0.001, 'weight_decay': 0.0005}
Test Accuracy: 0.20603015075376885
Fold 1/20
Best Validation Accuracy: 0.5333333333333333 at Epoch 0
Best hyperparameters: {'num_hidden': 32, 'lr': 0.001, 'weight_decay': 0.0005}
Test Accuracy: 0.20603015075376885
Fold 2/20
Best Validation Accuracy: 0.5333333333333333 at Epoch 0
Best hyperparameters: {'num_hidden': 32, 'lr': 0.001, 'weight_decay': 0.0005}
Test Accuracy: 0.20603015075376885
Fold 3/20
Best Validation Accuracy: 0.5333333333333333 at Epoch 0
Best hyperparameters: {'num_hidden': 32, 'lr': 0.001, 'weight_decay': 0.0005}
Test Accuracy: 0.20603015075376885
Fold 4/20
Best Validation Accuracy: 0.5333333333333333 at Epoch 0
Best hyperparameters: {'num_hidden': 32, 'lr': 0.001, 'weight_decay': 0.0005}
Test Accuracy: 0.20603015075376885
Fold 5/20
Best Validation Accuracy: 0.5333333333333333 at Epoch 0
Best hyperparameters: {'num_hidden': 32, 'lr': 0.

Grid Search Progress:  37%|███▋      | 10/27 [06:31<12:18, 43.42s/it]

Best Validation Accuracy: 0.5333333333333333 at Epoch 0
Best hyperparameters: {'num_hidden': 32, 'lr': 0.001, 'weight_decay': 0.0005}
Test Accuracy: 0.20603015075376885
Fold 1/20
Best Validation Accuracy: 0.5333333333333333 at Epoch 0
Best hyperparameters: {'num_hidden': 32, 'lr': 0.001, 'weight_decay': 0.0005}


RuntimeError: Error(s) in loading state_dict for GraphSage:
	size mismatch for conv1.lin_l.weight: copying a param with shape torch.Size([32, 1390]) from checkpoint, the shape in current model is torch.Size([64, 1390]).
	size mismatch for conv1.lin_l.bias: copying a param with shape torch.Size([32]) from checkpoint, the shape in current model is torch.Size([64]).
	size mismatch for conv1.lin_r.weight: copying a param with shape torch.Size([32, 1390]) from checkpoint, the shape in current model is torch.Size([64, 1390]).
	size mismatch for conv2.lin_l.weight: copying a param with shape torch.Size([32, 32]) from checkpoint, the shape in current model is torch.Size([64, 64]).
	size mismatch for conv2.lin_l.bias: copying a param with shape torch.Size([32]) from checkpoint, the shape in current model is torch.Size([64]).
	size mismatch for conv2.lin_r.weight: copying a param with shape torch.Size([32, 32]) from checkpoint, the shape in current model is torch.Size([64, 64]).
	size mismatch f

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from torch_geometric.nn import APPNP
from sklearn.model_selection import train_test_split, KFold
import numpy as np
import itertools
from tqdm import tqdm

class GraphSage(torch.nn.Module):
    def __init__(self, num_node_features, num_hidden, num_classes):
        super().__init__()
        self.appnp = APPNP(K=100, alpha=0.1, dropout=0, cached=True, add_self_loops=True, normalize=False)
        self.conv1 = SAGEConv(num_node_features, num_hidden)
        self.conv2 = SAGEConv(num_hidden, num_hidden)
        self.conv3 = SAGEConv(num_hidden, num_hidden)
        self.conv4 = SAGEConv(num_hidden, num_hidden)
        self.conv5 = SAGEConv(num_hidden, num_hidden)
        self.conv6 = SAGEConv(num_hidden, num_hidden)
        self.conv7 = SAGEConv(num_hidden, num_hidden)
        self.conv8 = SAGEConv(num_hidden, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.message_passing(x, edge_index, self.conv1)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.message_passing(x, edge_index, self.conv2)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.message_passing(x, edge_index, self.conv3)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.message_passing(x, edge_index, self.conv4)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.message_passing(x, edge_index, self.conv5)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.message_passing(x, edge_index, self.conv6)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.message_passing(x, edge_index, self.conv7)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.message_passing(x, edge_index, self.conv8)
        
        return F.log_softmax(x, dim=1)
    
    def message_passing(self, x, edge_index, conv):
        # Perform message passing with the given convolutional layer
        return conv(x, edge_index)
